In [384]:

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier

In [385]:
# Set pandas to display all columns
pd.set_option('display.max_columns', None)

In [386]:
import pandas as pd
import numpy as np

# data 5 is baddest and 9 is largest

df = pd.read_csv("odds_datas/test_9_odds_data.csv") # odds_data
df.rename(columns={"HT.1": 'AT'}, inplace=True)
df


,week,HT,AT,home,draw,away,over_1,under_1,over_2,under_2,over_3,under_3,over_4,under_4,H_score,A_score
0,21,EVE,BRN,2.88,3.14,2.59,1.29,3.62,1.93,1.87,3.42,1.31,7.99,1.08,3,2
1,21,MNC,ASV,1.57,4.03,6.02,1.20,4.56,1.69,2.17,2.84,1.43,6.09,1.12,1,1
2,21,FOR,FUL,2.29,3.02,3.53,1.33,3.35,2.07,1.75,3.76,1.27,9.27,1.06,2,3
3,21,MNU,WOL,1.77,3.33,5.32,1.22,4.33,1.74,2.09,2.87,1.42,6.36,1.12,1,0
4,21,SHU,LUT,2.24,3.07,3.58,1.32,3.37,2.07,1.76,3.76,1.27,9.29,1.06,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6085,21,EVE,WOL,2.45,3.19,3.03,1.29,3.61,1.94,1.86,3.45,1.31,8.00,1.08,0,0
6086,21,MNU,CRY,1.77,3.33,5.32,1.22,4.33,1.74,2.09,2.87,1.42,6.36,1.12,2,2
6087,21,SHU,BOU,2.43,3.24,3.01,1.33,3.31,2.07,1.76,3.85,1.26,9.22,1.06,0,0
6088,21,BRN,NWC,2.88,3.23,2.53,1.28,3.68,1.92,1.88,3.43,1.31,7.94,1.08,0,1


In [387]:
# path = "odds_datas"
# records = [
#     f"{path}/test_1_odds_data.csv", f"{path}/test_2_odds_data.csv", f"{path}/test_3_odds_data.csv"
# ]

## Feature Engineering

In [388]:

class Team:
    def __init__(self, name):
        self.name = name
        self.goals_for = 0
        self.goals_against = 0
        self.goal_difference = 0
        self.points = 0
        self.position = 0
        self.last_results = []  # List to store last results
        self.last_1_result = 0
        self.last_2_result = 0
        
def update_table(team_1, team_1_score, team_2, team_2_score):
    team1 = next(team for team in team_objects if team.name == team_1)
    team2 = next(team for team in team_objects if team.name == team_2)

    scores = [team_1_score, team_2_score]
    new_teams = [team1, team2]

    for i, team in enumerate(new_teams):
        team.goals_for += scores[i]
        team.goals_against += scores[1 - i]
        team.goal_difference = team.goals_for - team.goals_against
        team.points += 3 if scores[i] > scores[1 - i] else 1 if scores[i] == scores[1 - i] else 0
        team.last_results.append(1 if scores[i] > scores[1 - i] else 0 if scores[i] == scores[1 - i] else -1)
        team.last_1_result = team.last_results[-1]
        team.last_2_result = team.last_results[-2] if len(team.last_results) > 1 else 0

    table = sorted(team_objects, key=lambda x: (-x.points, x.name.lower()))

    for i, team in enumerate(table):
        team.position = i + 1

    return table


def get_record(table_objects):
    table_dict = dict()
    for team in table_objects:
        table_dict[team.name] = (
            team.points,
            team.position,
            team.last_1_result,
            # team.last_2_result,
            # team.goals_for,
            # team.goals_against,
            team.goal_difference
        )

    return table_dict


def add_features(data):
    dff = data.copy()
    # print(dff.shape)

    col_list = []

    for i, row in dff.iterrows():
        ht, hcs, at, acs = row["HT"], row["H_score"], row["AT"], row["A_score"]

        table = update_table(ht, hcs, at, acs)

        if row["week"] > 2:
            val = table_dict[ht] + table_dict[at]
        else:
            val = (0,) * 8
        col_list.append(val)
        # print(val)

        if (i+1) % 20 == 0:
            table_dict = get_record(table)

    return col_list


# ------------------------------------------------------------------------------------------------


# Define function to check home and away scores and return result
def get_result(row):
    # if row['H_score'] > row['A_score']:
    #     return 1  # home_win = 1
    # elif row['H_score'] < row['A_score']:
    #     return 2  # away_win = 2
    # else:
    #     return 0  # draw = 0
    
    if row["H_score"] + row["A_score"] < 4:
        return 1
    else:
        return 0
        

def feature_engineering(data):
    df_result = data.copy()
        
    # Apply function to each row of the dataframe to create a new column
    df_result['result'] = df_result.apply(get_result, axis=1)

    df_result = df_result[df_result['week'] > 2]  # remove rows with incomplete values

    # Create a LabelEncoder object
    label_encoder = LabelEncoder()

    # Fit the LabelEncoder on the unique team names
    label_encoder.fit(teams)

    # Convert the categorical features to numeric using label encoding
    df_result['HT_encoded'] = label_encoder.transform(df_result['HT'])
    df_result['AT_encoded'] = label_encoder.transform(df_result['AT'])

    # remove irrelevant features
    df_result = df_result.drop(["H_score", "A_score", "HT", "AT"], axis=1)

    df_result = df_result.reset_index(drop=True)  # reset the index values

    # # Scaling the dataset using standardization method
    # X = df_result.drop("result", axis=1)
    # y = df_result["result"]

    # # assume that X is your dataset with numerical features
    # scaler = StandardScaler()
    # X_scaled = scaler.fit_transform(X)

    # # # concatenate the scaled numerical features with the Target variable feature
    # data_preprocessed = pd.concat([pd.DataFrame(X_scaled, columns=X.columns), y], axis=1)

    return df_result


In [389]:
df_encoded = df.copy()
df_encoded = df_encoded.iloc[180: 5880]
df_encoded = df_encoded.reset_index(drop=True)

df_encoded

,week,HT,AT,home,draw,away,over_1,under_1,over_2,under_2,over_3,under_3,over_4,under_4,H_score,A_score
0,1,BRI,LUT,1.46,4.36,7.20,1.23,4.13,1.78,2.04,3.09,1.37,6.73,1.10,4,0
1,1,WHU,SHU,1.55,4.05,6.24,1.26,3.83,1.87,1.93,3.32,1.33,7.45,1.09,1,2
2,1,MNU,CRY,1.77,3.33,5.32,1.22,4.33,1.74,2.09,2.87,1.42,6.36,1.12,1,2
3,1,FUL,BOU,2.29,3.02,3.53,1.33,3.35,2.07,1.75,3.76,1.27,9.27,1.06,0,0
4,1,FOR,MNC,7.60,4.66,1.42,1.20,4.58,1.65,2.24,2.80,1.44,5.83,1.13,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5695,38,BUR,CHE,4.38,3.33,1.91,1.23,4.16,1.78,2.04,3.01,1.39,6.74,1.10,2,1
5696,38,MNC,BRN,1.47,4.38,7.03,1.18,4.86,1.61,2.31,2.64,1.48,5.44,1.15,2,0
5697,38,WOL,LIV,3.70,3.07,2.19,1.24,4.10,1.83,1.98,3.08,1.37,7.13,1.09,0,3
5698,38,EVE,LUT,1.95,3.29,4.25,1.33,3.33,2.08,1.75,3.83,1.26,9.27,1.06,2,0


In [390]:

df_new = []

split_num = 380

total_seasons = len(df_encoded) // split_num
# print("Total Seasons -", total_seasons)
start = 0
end = split_num
for _ in range(total_seasons):
    # print(start, end)
    df_temp = df_encoded.iloc[start: end].copy()

    teams = ['FOR', 'MNC', 'ASV', 'TOT', 'EVE', 'CHE', 'BRN', 'WHU', 'ARS', 'FUL', 
             'NWC', 'BOU', 'BUR', 'LIV', 'WOL', 'MNU', 'LUT', 'SHU', 'BRI', 'CRY']

    # teams = ['FOR', 'MNC', 'ASV', 'TOT', 'EVE', 'CHE', 'BRN', 'WHU', 'ARS', 'FUL',
    #          'NWC', 'BOU', 'LEI', 'LIV', 'WOL', 'MNU', 'LEE', 'SOU', 'BRI', 'CRY']
            
    # create a new empty league table to record scores & points
    team_objects = [Team(team) for team in teams]


    # adds scores & points from each league to the dataframe
    new_df = add_features(df_temp)
    # print(new_df)
    df_new += new_df  # add each dataframe to a list

    start += split_num
    end += split_num

cols = ["h_pts", "h_pos", "HL1R", "h_gd",
        "a_pts", "a_pos", "AL1R", "a_gd"]
records_df = pd.DataFrame(df_new, columns=cols)
records_df = pd.concat([df_encoded, records_df], axis=1)
# records_df

process_df = feature_engineering(records_df)

In [391]:
process_df

,week,home,draw,away,over_1,under_1,over_2,under_2,over_3,under_3,over_4,under_4,h_pts,h_pos,HL1R,h_gd,a_pts,a_pos,AL1R,a_gd,result,HT_encoded,AT_encoded
0,3,2.29,3.00,3.54,1.29,3.60,1.97,1.83,3.47,1.31,8.41,1.07,3,12,1,-2,6,3,1,2,1,19,7
1,3,1.95,3.29,4.25,1.33,3.33,2.08,1.75,3.83,1.26,9.27,1.06,1,17,-1,-1,4,6,0,1,1,10,16
2,3,8.62,4.90,1.36,1.21,4.45,1.67,2.20,2.85,1.42,5.92,1.13,1,15,-1,-1,0,20,-1,-4,1,5,13
3,3,1.95,3.29,4.25,1.33,3.33,2.08,1.75,3.83,1.26,9.27,1.06,1,16,0,-1,0,19,-1,-5,0,8,12
4,3,2.48,3.20,2.98,1.26,3.83,1.88,1.92,3.26,1.34,7.53,1.09,3,7,-1,3,6,5,1,5,1,3,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5395,38,4.38,3.33,1.91,1.23,4.16,1.78,2.04,3.01,1.39,6.74,1.10,36,18,0,-17,47,11,-1,7,1,5,6
5396,38,1.47,4.38,7.03,1.18,4.86,1.61,2.31,2.64,1.48,5.44,1.15,67,2,0,30,53,8,0,4,1,13,4
5397,38,3.70,3.07,2.19,1.24,4.10,1.83,1.98,3.08,1.37,7.13,1.09,40,15,0,-10,60,4,1,11,1,19,11
5398,38,1.95,3.29,4.25,1.33,3.33,2.08,1.75,3.83,1.26,9.27,1.06,31,19,1,-25,39,16,-1,-23,1,8,12


In [392]:
process_df.isnull().sum()

week          0
home          0
draw          0
away          0
over_1        0
under_1       0
over_2        0
under_2       0
over_3        0
under_3       0
over_4        0
under_4       0
h_pts         0
h_pos         0
HL1R          0
h_gd          0
a_pts         0
a_pos         0
AL1R          0
a_gd          0
result        0
HT_encoded    0
AT_encoded    0
dtype: int64

In [393]:
duplicates = process_df[process_df.duplicated()].shape[0]

print("Total Duplicate Values:", duplicates)

# # Dropping duplicate values
# df_encoded.drop_duplicates(inplace=True)
# df_encoded.head()

Total Duplicate Values: 0


In [394]:

# over_number = 2
# lable = f"over_{over_number}_result"

# df["total"] = df["H_score"] + df["A_score"]
# df[lable] = np.where(df['total'] > over_number, 1, 0)
# # df = df.drop(["H_score", "A_score", "total"], axis=1)

# df_encoded = df.copy()
# df_encoded = df_encoded.drop(["HT", "AT", "H_score", "A_score", "total"], axis=1)

# # df_encoded = pd.get_dummies(df, columns=['HT', 'AT'])
# # df_encoded.replace({True: 1, False: 0}, inplace=True)

# df_encoded

# Model Building

In [395]:
lable = "result"

# Split the data into features and target variable
X = process_df.drop(lable, axis=1)
y = process_df[lable]

# Standardize the features using StandardScaler before splitting
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Create a dictionary to store the models and their names
models = {
    'Random Forest': RandomForestClassifier(random_state=42),
    'Logistic Regression': LogisticRegression(random_state=42),
    'Support Vector Machine': SVC(random_state=42),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Gaussian Naive Bayes': GaussianNB(),
    'Multi-layer Perceptron': MLPClassifier(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'AdaBoost': AdaBoostClassifier(random_state=42),
    'XGBoost': XGBClassifier(random_state=42)
}

# Test each model and print the accuracy and F1 score
for model_name, model in models.items():
    # Fit the model on the training data
    model.fit(X_train, y_train)
    
    # Make predictions on the test data
    y_pred = model.predict(X_test)
    
    # Calculate the accuracy and F1 score of the model
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    # Print the results
    print(f"Model: {model_name}")
    print('Accuracy:', accuracy)
    print('F1 score:', f1)
    print('-' * 30, "\n")

Model: Random Forest
Accuracy: 0.6731481481481482
F1 score: 0.6013115745003936
------------------------------ 

Model: Logistic Regression
Accuracy: 0.7037037037037037
F1 score: 0.5813204508856683
------------------------------ 



c:\Users\CJay\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Model: Support Vector Machine
Accuracy: 0.7037037037037037
F1 score: 0.5813204508856683
------------------------------ 

Model: K-Nearest Neighbors
Accuracy: 0.6268518518518519
F1 score: 0.5991061922560393
------------------------------ 

Model: Decision Tree
Accuracy: 0.5861111111111111
F1 score: 0.589786590511229
------------------------------ 

Model: Gaussian Naive Bayes
Accuracy: 0.600925925925926
F1 score: 0.5956197289082809
------------------------------ 



c:\Users\CJay\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Model: Multi-layer Perceptron
Accuracy: 0.6851851851851852
F1 score: 0.5992256486083646
------------------------------ 

Model: Gradient Boosting
Accuracy: 0.6907407407407408
F1 score: 0.5797912327324092
------------------------------ 

Model: AdaBoost
Accuracy: 0.700925925925926
F1 score: 0.5881919138624119
------------------------------ 

Model: XGBoost
Accuracy: 0.6481481481481481
F1 score: 0.6085864172947673
------------------------------ 



In [396]:
# Model: Random Forest
# Accuracy: 0.5101851851851852
# F1 score: 0.507061670435882

# Model: XGBoost
# Accuracy: 0.5342592592592592
# F1 score: 0.5323126041908798

In [397]:
# Model: Random Forest
# Accuracy: 0.590311986863711
# F1 score: 0.5817079497014717
# ------------------------------ 

# Model: Logistic Regression
# Accuracy: 0.7003284072249589
# F1 score: 0.5769001751452342
# ------------------------------ 

# Model: Support Vector Machine
# Accuracy: 0.7003284072249589
# F1 score: 0.5769001751452342
# ------------------------------ 

# Model: K-Nearest Neighbors
# Accuracy: 0.6182266009852216
# F1 score: 0.5929422105742812
# ------------------------------ 

# Model: Decision Tree
# Accuracy: 0.5919540229885057
# F1 score: 0.5711342576228249
# ------------------------------ 

# Model: Gaussian Naive Bayes
# Accuracy: 0.6116584564860427
# F1 score: 0.6019015252836162
# ------------------------------ 

# Model: Multi-layer Perceptron
# Accuracy: 0.7003284072249589
# F1 score: 0.5769001751452342
# ------------------------------ 

# Model: Gradient Boosting
# Accuracy: 0.6978653530377669
# F1 score: 0.583027435177162
# ------------------------------ 

# Model: AdaBoost
# Accuracy: 0.7003284072249589
# F1 score: 0.5769001751452342
# ------------------------------ 

# Model: XGBoost
# Accuracy: 0.6182266009852216
# F1 score: 0.5889138400869722
# ------------------------------ 

In [398]:
# Model: Random Forest
# Accuracy: 0.5073891625615764
# F1 score: 0.5062100408762183
# ------------------------------ 

# Model: Logistic Regression
# Accuracy: 0.5287356321839081
# F1 score: 0.4957123651030123
# ------------------------------ 

# Model: Support Vector Machine
# Accuracy: 0.5336617405582923
# F1 score: 0.466537283013197
# ------------------------------ 

# Model: K-Nearest Neighbors
# Accuracy: 0.5016420361247947
# F1 score: 0.49909559609812915
# ------------------------------ 

# Model: Decision Tree
# Accuracy: 0.4967159277504105
# F1 score: 0.4968126153522162
# ------------------------------ 

# Model: Gaussian Naive Bayes
# Accuracy: 0.5262725779967159
# F1 score: 0.5233768995360785
# ------------------------------ 

# Model: Multi-layer Perceptron
# Accuracy: 0.5344827586206896
# F1 score: 0.5161139404531753
# ------------------------------ 

# Model: Gradient Boosting
# Accuracy: 0.5402298850574713
# F1 score: 0.5336910561690722
# ------------------------------ 

# Model: AdaBoost
# Accuracy: 0.5451559934318555
# F1 score: 0.5341392606141279
# ------------------------------ 

# Model: XGBoost
# Accuracy: 0.5090311986863711
# F1 score: 0.5057990955051651
# ------------------------------ 